# STEREOSET
**StereoSet** è un dataset progettato per misurare e rilevare i **bias nei modelli di linguaggio** rispetto a diverse categorie, come **genere, razza, religione, e orientamento sessuale**. Il dataset è suddiviso in **tre diverse sezioni** — **Intrasentence**, **Intersentence**, e **Commonsense** — che servono a valutare i bias in contesti differenti e con diversi tipi di frasi.

### Le sezioni di StereoSet:

#### 1. **Intrasentence**:
- **Descrizione**: Questa sezione contiene frasi singole in cui il bias si trova **all'interno della frase stessa**. L'obiettivo è valutare se i modelli di linguaggio **preferiscono stereotipi** o frasi che trasmettono bias all'interno di una singola frase, rispetto a opzioni neutre o contrarie.
- **Struttura**: Ogni esempio contiene una frase "context" seguita da tre possibili completamenti:
  1. **Stereotypical (stereotipico)**: Una continuazione che riflette un bias o stereotipo.
  2. **Anti-stereotypical (anti-stereotipico)**: Una continuazione che contrasta lo stereotipo.
  3. **Unrelated (non correlato)**: Una continuazione che non è rilevante o non si collega direttamente alla frase.

**Esempio**:
- **Contesto**: "John loves fixing cars. He must be a..."
  - **Stereotypical completamento**: "mechanic."
  - **Anti-stereotypical completamento**: "teacher."
  - **Unrelated completamento**: "banana."

In questo esempio, il modello viene testato per vedere se predilige un completamento stereotipico, come "mechanic" (stereotipo di genere che associa gli uomini ai meccanici), rispetto a "teacher" o "banana".

#### 2. **Intersentence**:
- **Descrizione**: Questa sezione contiene **due frasi consecutive**, in cui il bias è presente **nella relazione tra le due frasi**. Qui, il bias può essere testato rispetto al modo in cui il modello collega il contesto di una frase alla successiva.
- **Struttura**: Come in Intrasentence, anche qui ci sono tre opzioni per la seconda frase:
  1. **Stereotypical**: Una seconda frase che segue uno stereotipo.
  2. **Anti-stereotypical**: Una seconda frase che contrasta lo stereotipo.
  3. **Unrelated**: Una seconda frase non collegata.

**Esempio**:
- **Frase 1**: "Jane works as a software engineer."
  - **Stereotypical seconda frase**: "She must be bad at coding."
  - **Anti-stereotypical seconda frase**: "She is great at coding."
  - **Unrelated seconda frase**: "She loves painting on weekends."

In questo caso, il modello è testato per verificare se associa in modo stereotipato una donna ingegnere a una valutazione negativa delle sue capacità tecniche.

#### 3. **Commonsense**:
- **Descrizione**: Questa sezione valuta le conoscenze generali e il ragionamento **basato sul buon senso**, senza necessariamente includere stereotipi. Questo serve a garantire che il modello non solo eviti i bias, ma che abbia anche una buona comprensione del **buon senso**.
- **Struttura**: Ogni esempio commonsense presenta una frase e tre possibili completamenti. Qui, l'obiettivo è vedere se il modello può scegliere il completamento che ha più senso a livello logico.
  1. **True**: Un completamento che ha senso dal punto di vista del buon senso.
  2. **False**: Un completamento che non ha senso.
  3. **Unrelated**: Un completamento che non è rilevante per la frase.

**Esempio**:
- **Frase**: "The sun rises in the..."
  - **True completamento**: "east."
  - **False completamento**: "west."
  - **Unrelated completamento**: "ocean."

In questo caso, il modello deve dimostrare di avere un buon livello di conoscenza del mondo reale, scegliendo la risposta corretta ("east") rispetto a risposte errate o non correlate.

### Obiettivo delle tre sezioni

- **Intrasentence** e **Intersentence** sono progettate per rilevare **bias e stereotipi** nelle preferenze del modello, testando se è incline a scegliere frasi stereotipate rispetto a frasi anti-stereotipate o neutrali.
- **Commonsense** serve come controllo per garantire che un modello non stia solo evitando bias, ma che possieda anche una buona comprensione del **ragionamento basato sul buon senso**. Questo è importante perché un modello che evita i bias, ma non ha un buon ragionamento, non sarà utile in molte applicazioni pratiche.

### Dettagli sui tipi di bias rilevati

StereoSet classifica i bias in **quattro categorie principali**:
1. **Genere**: Bias legati a ruoli o caratteristiche di genere (es. uomini associati a lavori tecnici, donne a ruoli di cura).
2. **Razza**: Bias basati su stereotipi razziali (es. associando determinate capacità o comportamenti a un gruppo etnico specifico).
3. **Religione**: Bias riguardanti stereotipi religiosi (es. associando determinate credenze o comportamenti a una religione).
4. **Orientamento sessuale**: Bias verso persone di orientamenti sessuali diversi (es. stereotipi sull'omosessualità o l'eterosessualità).

### Utilizzo del dataset

Puoi usare il dataset per **valutare i modelli di linguaggio** e verificare se tendono a favorire risposte stereotipate o se mostrano segni di bias nei loro completamenti di frasi. La struttura del dataset ti permette di confrontare le risposte stereotipate con quelle anti-stereotipate e di misurare il livello di bias presente nel modello.

In [1]:
import os
import spacy
import pandas as pd
import requests
import json
import numpy as np
import time
from datasets import load_dataset
from neo4j import GraphDatabase
from nltk.corpus import wordnet as wn
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Scarica il corpus WordNet
nltk.download('wordnet')
nltk.download('omw-1.4')
# Assicuriamoci di scaricare le stopwords di NLTK (solo la prima volta)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/danilogiovannico/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/danilogiovannico/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danilogiovannico/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# File di cache
CACHE_FILE_ENTITIES = "cache/entity_cache_populate_neo4j.json"
CACHE_FILE_PHRASES = "cache/phrase_cache_populate_neo4j.json"

# Funzione per caricare la cache dal disco
def load_cache(CACHE_FILE):
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as cache_file:
            return json.load(cache_file)
    return {}

# Funzione per salvare la cache sul disco
def save_cache(CACHE_FILE, cache):
    with open(CACHE_FILE, 'w') as cache_file:
        json.dump(cache, cache_file, indent=4)

# Carica la cache prima di popolare Neo4j
entity_cache = load_cache(CACHE_FILE_ENTITIES)
phrase_cache = load_cache(CACHE_FILE_PHRASES)

In [3]:
def remove_stopwords_and_lemmatizer(text, remove_stopwords=True, lemmatize=True):
    # Converti la frase in un oggetto SpaCy per l'analisi
    doc = nlp(text)
    
    # Rimuove le stopwords e/o lemmatizza in base ai parametri
    processed_tokens = []
    for token in doc:
        if remove_stopwords and token.text.lower() in stop_words:
            continue
        if lemmatize:
            processed_tokens.append(token.lemma_)
        else:
            processed_tokens.append(token.text)
    
    # Unisce la frase pre-elaborata
    return " ".join(processed_tokens)

In [4]:
# Configura la connessione a Neo4j
class Neo4jHandler:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_node_if_not_exists(self, term, roles, wikidata_description=None, wordnet_description=None):
        """
        Crea un nodo se non esiste già. Usa il termine come ID del nodo.
        Se il nodo esiste già, aggiorna la sua descrizione se disponibile una nuova da Wikidata o WordNet.
        """
        with self.driver.session() as session:
            # Concateniamo le descrizioni se disponibili senza aggiungere prefissi
            descriptions = []
            if wikidata_description:
                descriptions.append(wikidata_description)  # Descrizione da Wikidata
            if wordnet_description:
                descriptions.append(wordnet_description)  # Descrizione da WordNet
            final_description = " | ".join(descriptions) if descriptions else ""
    
            # Primo step: creiamo o aggiorniamo il nodo. Se esiste, aggiorniamo la descrizione.
            query_merge = """
            MERGE (n:Entity {id: $term})
            ON CREATE SET n.label = $label, n.description = $description
            ON MATCH SET n.description = CASE 
                WHEN $description <> '' THEN $description 
                ELSE n.description 
            END
            RETURN n
            """
            session.run(query_merge, term=term, label='|'.join(roles), description=final_description)
    
            # Secondo step: aggiorniamo la label solo se non è già inclusa
            for role in roles:
                query_update_label = """
                MATCH (n:Entity {id: $term})
                WHERE NOT n.label CONTAINS $role
                SET n.label = n.label + '|' + $role
                RETURN n
                """
                session.run(query_update_label, term=term, role=role)

    def create_relationship_if_not_exists(self, term_id, related_entity_id, relationship_type):
        """
        Crea una relazione tra due nodi solo se non esiste già.
        """
        with self.driver.session() as session:
            query = f"""
            MATCH (a:Entity {{id: $term_id}}), (b:Entity {{id: $related_entity_id}})
            MERGE (a)-[r:{relationship_type}]->(b)
            RETURN r
            """
            session.run(query, term_id=term_id, related_entity_id=related_entity_id)

    def verb_exists_in_neo4j(self, verbo):
        """
        Controlla se un nodo per il verbo (lemma) esiste già nel grafo Neo4j.
        """
        with self.driver.session() as session:
            query = """
            MATCH (n:Entity {id: $verbo})
            RETURN n LIMIT 1
            """
            result = session.run(query, verbo=verbo)
            return result.single() is not None

    def get_entity_info_from_neo4j(self, term):
        query = """
        MATCH (n:Entity {id: $term})
        OPTIONAL MATCH (n)-[:INSTANCE_OF]->(instance)
        OPTIONAL MATCH (n)-[:SUBCLASS_OF]->(subclass)
        RETURN n.description AS description, collect(DISTINCT instance.id) AS instance_of, collect(DISTINCT subclass.id) AS subclass_of
        """
        with self.driver.session() as session:
            result = session.run(query, term=term).single()
        
        return {
            'description': result['description'] if result and result['description'] else [],
            'instance_of': result['instance_of'] if result and result['instance_of'] else [],
            'subclass_of': result['subclass_of'] if result and result['subclass_of'] else []
        }


    def get_verb_info_from_neo4j(self, term):
        query = """
        MATCH (n:Entity {id: $term})
        OPTIONAL MATCH (n)-[:SYNONYM_OF]->(synonym)
        OPTIONAL MATCH (n)-[:ANTONYM_OF]->(antonym)
        RETURN n.description AS description, collect(DISTINCT synonym.id) AS synonyms, collect(DISTINCT antonym.id) AS antonyms
        """
        with self.driver.session() as session:
            result = session.run(query, term=term).single()
        
        return {
            'description': result['description'] if result and result['description'] else [],
            'synonyms': result['synonyms'] if result and result['synonyms'] else [],
            'antonyms': result['antonyms'] if result and result['antonyms'] else []
        }

    def get_adj_adv_info_from_neo4j(self, term):
        query = """
        MATCH (n:Entity {id: $term})
        OPTIONAL MATCH (n)-[:SYNONYM_OF]->(synonym)
        OPTIONAL MATCH (n)-[:ANTONYM_OF]->(antonym)
        RETURN collect(DISTINCT synonym.id) AS synonyms, collect(DISTINCT antonym.id) AS antonyms
        """
        with self.driver.session() as session:
            result = session.run(query, term=term).single()
        return {
            'synonyms': result['synonyms'],
            'antonyms': result['antonyms']
        }

In [5]:
WIKIDATA_API_URL = "https://www.wikidata.org/w/api.php"

def search_wikidata(term, limit=20, sleep=0):
    """
    Cerca entità su Wikidata e ritorna id, label e descrizione.
    """
    params = {
        "action": "wbsearchentities",
        "search": term,
        "language": "en",
        "format": "json",
        "limit": limit
    }
    time.sleep(sleep)
    response = requests.get(WIKIDATA_API_URL, params=params)
    results = response.json().get("search", [])
    
    entities = []
    for entity in results:
        entities.append({
            "id": entity.get("id"),
            "label": entity.get("label"),
            "description": entity.get("description")
        })
    
    return entities

def get_entity_data(entity_id, sleep=0.1):
    """
    Ottiene le proprietà 'instance of' e 'subclass of' per una determinata entità.
    """
    params = {
        "action": "wbgetentities",
        "ids": entity_id,
        "props": "claims",
        "format": "json"
    }
    time.sleep(sleep)
    response = requests.get(WIKIDATA_API_URL, params=params)
    entity_data = response.json().get("entities", {}).get(entity_id, {})
    claims = entity_data.get("claims", {})
    
    instance_of = claims.get("P31", [])
    subclass_of = claims.get("P279", [])
    
    return instance_of, subclass_of

def extract_labels_descriptions(property_claims, sleep=0):
    """
    Estrae etichette e descrizioni per ogni entità associata a 'instance of' o 'subclass of'.
    """
    ids = [claim["mainsnak"]["datavalue"]["value"]["id"] for claim in property_claims if "mainsnak" in claim and "datavalue" in claim["mainsnak"]]
    
    if not ids:
        return []
    
    ids_str = "|".join(ids)
    
    params = {
        "action": "wbgetentities",
        "ids": ids_str,
        "props": "labels|descriptions",
        "languages": "en",  # Otteniamo etichette e descrizioni in inglese
        "format": "json"
    }
    time.sleep(sleep)
    response = requests.get(WIKIDATA_API_URL, params=params)
    entities = response.json().get("entities", {})
    
    results = []
    for entity_id, entity_info in entities.items():
        # Estrai la descrizione dall'entità, se disponibile
        label = entity_info.get("labels", {}).get("en", {}).get("value", "")
        description = entity_info.get("descriptions", {}).get("en", {}).get("value", "")
        
        # Se non c'è descrizione, imposta un valore predefinito o lascialo vuoto
        description = description if description else ""
                
        results.append({
            "id": entity_id,
            "label": label,
            "description": description  # Usa la descrizione estratta da Wikidata
        })
    
    return results

def save_to_neo4j(neo4j_handler, term, instance_of, subclass_of, wordnet_description=None):
    """
    Salva i dati estratti su Neo4j e crea relazioni 'instance of' e 'subclass of'.
    Usa descrizioni da Wikidata e WordNet, se disponibili.
    """
    # Ottieni la descrizione da Wikidata se disponibile
    wikidata_description = None
    if instance_of:
        wikidata_description = instance_of[0]["description"]  # Usa direttamente la descrizione da Wikidata
    elif subclass_of:
        wikidata_description = subclass_of[0]["description"]  # Usa direttamente la descrizione da Wikidata

    # Se esiste una descrizione da Wikidata, usiamola
    # Aggiungiamo anche la descrizione da WordNet se disponibile
    final_description = ""
    if wikidata_description and wikidata_description != "No description available":
        final_description += wikidata_description
    if wordnet_description:
        if final_description:
            final_description += " | "  # Concatena le descrizioni se entrambe esistono
        final_description += wordnet_description

    # Passiamo la descrizione combinata (senza prefissi) al nodo
    neo4j_handler.create_node_if_not_exists(term, ['ENTITY'], final_description)
    
    # Inserisci i nodi e le relazioni 'instance of'
    for instance in instance_of:
        neo4j_handler.create_node_if_not_exists(instance["label"], ['INSTANCE'], instance["description"])
        neo4j_handler.create_relationship_if_not_exists(term, instance["label"], "INSTANCE_OF")
    
    # Inserisci i nodi e le relazioni 'subclass of'
    for subclass in subclass_of:
        neo4j_handler.create_node_if_not_exists(subclass["label"], ['SUBCLASS'], subclass["description"])
        neo4j_handler.create_relationship_if_not_exists(term, subclass["label"], "SUBCLASS_OF")

def enrich_with_synonyms_and_antonyms(neo4j_handler, term):
    """
    Arricchisce il nodo principale con tutti i sinonimi e contrari trovati tramite WordNet.
    I sinonimi e contrari vengono tutti collegati direttamente al nodo principale.
    """
    synsets = wn.synsets(term)

    if not synsets:
        print(f"Nessun synset trovato per il termine '{term}'")
        return

    processed_synonyms = set()  # Cache per tenere traccia dei sinonimi già elaborati

    for synset in synsets:
        lemmas = synset.lemmas()
        
        for lemma in lemmas:
            synonym = lemma.name()
            
            # Assicuriamoci di non creare duplicati
            if synonym != term and synonym not in processed_synonyms:
                processed_synonyms.add(synonym)
                # Ottieni la definizione di WordNet per il sinonimo
                description = synset.definition() if synset else f"Synonym of {term}"
                #print(f"Processo sinonimo: {synonym}")
                # Crea il nodo per il sinonimo SENZA label
                neo4j_handler.create_node_if_not_exists(synonym, [], description)
                neo4j_handler.create_relationship_if_not_exists(term, synonym, "SYNONYM_OF")
            
            antonyms = lemma.antonyms()
            
            for antonym in antonyms:
                antonym_name = antonym.name()
                if antonym_name != term and antonym_name not in processed_synonyms:
                    processed_synonyms.add(antonym_name)
                    # Ottieni la definizione di WordNet per il contrario
                    antonym_synset = wn.synsets(antonym_name)
                    description = antonym_synset[0].definition() if antonym_synset else f"Antonym of {term}"
                    #print(f"Processo antonimo: {antonym_name}")
                    # Crea il nodo per il contrario SENZA label
                    neo4j_handler.create_node_if_not_exists(antonym_name, [], description)
                    neo4j_handler.create_relationship_if_not_exists(term, antonym_name, "ANTONYM_OF")

def process_verb_with_wordnet(neo4j_handler, verbo, roles):
    """
    Elabora i verbi (lemmi) arricchendoli con sinonimi e contrari da WordNet.
    Gestisce anche le relazioni se il verbo esiste già.
    """
    synsets = wn.synsets(verbo, pos=wn.VERB)
    description = synsets[0].definition() if synsets else f"Verb: {verbo}"

    # Controlliamo se il verbo esiste già
    if neo4j_handler.verb_exists_in_neo4j(verbo):
        #print(f"Verbo '{verbo}' già presente nel grafo, creando comunque le relazioni.")
        # Anche se il nodo esiste, creiamo le relazioni e aggiorniamo i ruoli
        neo4j_handler.create_node_if_not_exists(verbo, roles, description)
        enrich_with_synonyms_and_antonyms(neo4j_handler, verbo)
        return

    # Aggiungi il verbo come nodo al grafo
    neo4j_handler.create_node_if_not_exists(verbo, roles, description)
    
    # Aggiungi sinonimi e contrari per il verbo
    enrich_with_synonyms_and_antonyms(neo4j_handler, verbo)


def process_entity(term, neo4j_handler, limit=50):
    """
    Processa un'entità, arricchendola con dati da Wikidata, e aggiunge sinonimi e contrari da WordNet.
    """
    # Ottieni i dati da Wikidata
    entities = search_wikidata(term, limit=limit)
    
    if not entities:
        print(f"Nessun risultato trovato per il termine '{term}'")
        return
    
    instance_of_entities = []
    subclass_of_entities = []

    for entity_data in entities:
        entity_id = entity_data["id"]
        instance_of_claims, subclass_of_claims = get_entity_data(entity_id)
        instance_of_entities.extend(extract_labels_descriptions(instance_of_claims))
        subclass_of_entities.extend(extract_labels_descriptions(subclass_of_claims))
    
    # Ottieni la definizione di WordNet se esiste
    wordnet_synsets = wn.synsets(term)
    wordnet_description = wordnet_synsets[0].definition() if wordnet_synsets else None

    # Salva i dati su Neo4j con entrambe le descrizioni
    save_to_neo4j(neo4j_handler, term, instance_of_entities, subclass_of_entities, wordnet_description)

    # Aggiungi sinonimi e contrari da WordNet
    enrich_with_synonyms_and_antonyms(neo4j_handler, term)

# Funzione per estrarre ruoli sintattici, NER e relazioni SVO
def extract_syntactic_roles_and_ner(text):
    """
    Estrae soggetti, predicati, oggetti, aggettivi, avverbi, sostantivi, entità nominate e relazioni SVO da un testo.
    """
    doc = nlp(text)
    
    soggetti = []
    predicati = []
    oggetti = []
    aggettivi = []
    avverbi = []
    sostantivi = []
    entita_nominate = []
    relazioni_svo = []

    for token in doc:
        if token.dep_ == "nsubj":
            soggetti.append(token.text)
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            predicati.append(token.text)
        if token.dep_ == "dobj":
            oggetti.append(token.text)
        if token.pos_ == "ADJ":
            aggettivi.append(token.text)
        if token.pos_ == "ADV":
            avverbi.append(token.text)
        if token.pos_ == "NOUN":
            sostantivi.append(token.text)

    for token in doc:
        if token.dep_ == "ROOT":
            soggetto = None
            oggetto = None
            pred_aggettivo = None

            for child in token.children:
                if child.dep_ in ["nsubj", "nsubjpass", "csubj"]:
                    soggetto = child.text

            for child in token.children:
                if child.dep_ in ["dobj", "pobj", "iobj"]:
                    oggetto = child.text
                if child.dep_ in ["acomp", "attr", "oprd"]:
                    pred_aggettivo = child.text

            if soggetto and oggetto:
                relazione = {"soggetto": soggetto, "verbo": token.text, "oggetto": oggetto}
                if relazione not in relazioni_svo:
                    relazioni_svo.append(relazione)
            elif soggetto and pred_aggettivo:
                relazione = {"soggetto": soggetto, "verbo": token.text, "predicato": pred_aggettivo}
                if relazione not in relazioni_svo:
                    relazioni_svo.append(relazione)

    entita_nominate = [{"label": ent.text, "type": ent.label_} for ent in doc.ents]

    return soggetti, predicati, oggetti, aggettivi, avverbi, sostantivi, entita_nominate, relazioni_svo


# Funzione per arricchire ogni frase e popolare Neo4j
def enrich_sentence(row, enable_neo4j_population=True, enable_context=False):
    sentence = row['Frase']
    soggetti, predicati, oggetti, aggettivi, avverbi, sostantivi, entita_nominate, relazioni_svo = extract_syntactic_roles_and_ner(sentence)

    if enable_neo4j_population == True:
        populate_neo4j(sentence, soggetti, predicati, oggetti, aggettivi, avverbi, sostantivi, entita_nominate, relazioni_svo)
        context = row['Contesto']
        if 'BLANK' not in context:
            soggetti_contesto, predicati_contesto, oggetti_contesto, aggettivi_contesto, avverbi_contesto, sostantivi_contesto, entita_nominate_contesto, relazioni_svo_contesto = extract_syntactic_roles_and_ner(context)
            populate_neo4j(context, soggetti_contesto, predicati_contesto, oggetti_contesto, aggettivi_contesto, avverbi_contesto, sostantivi_contesto, entita_nominate_contesto, relazioni_svo_contesto)
    
    return pd.Series({
        'Soggetti': soggetti,
        'Predicati': predicati,
        'Oggetti': oggetti,
        'Aggettivi': aggettivi,
        'Avverbi': avverbi,
        'Sostantivi': sostantivi,
        #'Entità Nominate': entita_nominate,
        'Relazioni SVO': relazioni_svo
    })


def populate_neo4j(phrase, soggetti, predicati, oggetti, aggettivi, avverbi, sostantivi, entita_nominate, relazioni_svo):
    if phrase not in phrase_cache:
        print(f"Inizio popolamento del database Neo4j per la frase: '{phrase}'")
        # Inserimento delle entità nominate
        for ent in entita_nominate:
            term = ent['label']
            if term not in entity_cache:
                process_entity(term, neo4j_handler)  # Usa il normale metodo process_entity
                entity_cache[term] = True  # Memorizza l'entità nella cache
                save_cache(CACHE_FILE_ENTITIES, entity_cache)  # Salva la cache su disco
    
        # Inserimento dei soggetti
        for soggetto in soggetti:
            roles = ['SUBJECT']
            if soggetto in sostantivi:
                roles.append('NOUN')
            if soggetto not in entity_cache:
                process_entity(soggetto, neo4j_handler)  # Usa il normale metodo process_entity
                entity_cache[soggetto] = True  # Memorizza il soggetto nella cache
                save_cache(CACHE_FILE_ENTITIES, entity_cache)  # Salva la cache su disco
    
        # Inserimento dei predicati
        for predicato in predicati:
            roles = ['PREDICATE']
            if predicato not in entity_cache:
                process_verb_with_wordnet(neo4j_handler, predicato, roles)  # Usa process_verb_with_wordnet
                entity_cache[predicato] = True  # Memorizza il predicato nella cache
                save_cache(CACHE_FILE_ENTITIES, entity_cache)  # Salva la cache su disco
    
        # Inserimento degli oggetti
        for oggetto in oggetti:
            roles = ['OBJECT']
            if oggetto in sostantivi:
                roles.append('NOUN')
            if oggetto not in entity_cache:
                process_entity(oggetto, neo4j_handler)  # Usa il normale metodo process_entity
                entity_cache[oggetto] = True  # Memorizza l'oggetto nella cache
                save_cache(CACHE_FILE_ENTITIES, entity_cache)  # Salva la cache su disco
    
        # Inserimento degli aggettivi
        for aggettivo in aggettivi:
            if aggettivo not in entity_cache:
                process_entity(aggettivo, neo4j_handler)  # Usa il normale metodo process_entity
                entity_cache[aggettivo] = True  # Memorizza l'aggettivo nella cache
                save_cache(CACHE_FILE_ENTITIES, entity_cache)  # Salva la cache su disco
    
        # Inserimento degli avverbi
        for avverbio in avverbi:
            if avverbio not in entity_cache:
                process_entity(avverbio, neo4j_handler)  # Usa il normale metodo process_entity
                entity_cache[avverbio] = True  # Memorizza l'avverbio nella cache
                save_cache(CACHE_FILE_ENTITIES, entity_cache)  # Salva la cache su disco
    
        # Inserimento delle relazioni SVO
        for relazione in relazioni_svo:
            soggetto = relazione.get('soggetto')
            verbo = relazione.get('verbo')
            oggetto = relazione.get('oggetto')
            predicato = relazione.get('predicato')
            # Processiamo il soggetto
            process_entity(soggetto, neo4j_handler)
            process_verb_with_wordnet(neo4j_handler, verbo, ['PREDICATE'])

            # Creiamo la relazione tra soggetto e verbo (chi compie l'azione)
            neo4j_handler.create_relationship_if_not_exists(soggetto, verbo, "PERFORMS")
            
            # Se c'è un oggetto, processiamolo
            if oggetto:
                process_entity(oggetto, neo4j_handler)
                neo4j_handler.create_relationship_if_not_exists(verbo, oggetto, "APPLIES_TO")
            elif predicato:
                process_entity(predicato, neo4j_handler)
                neo4j_handler.create_relationship_if_not_exists(verbo, predicato, "APPLIES_TO")

        phrase_cache[phrase] = True
        save_cache(CACHE_FILE_PHRASES, phrase_cache)  # Salva la cache su disco
        print("Frase analizzata.")

In [6]:
def generate_sbert_embeddings(documents, use_batching=True):
    if use_batching:
        embeddings = sbert_model.encode(documents, convert_to_tensor=True)
        return embeddings.cpu().numpy()  # Ritorna embeddings come numpy array
    else:
        return np.vstack([sbert_model.encode(doc) for doc in documents])

def process_description_embeddings(descriptions, model):
    """Genera un embedding per ogni descrizione individualmente usando SentenceTransformer"""
    
    # Assicurati che descriptions sia una lista
    if isinstance(descriptions, str):
        descriptions = [descriptions]  # Se è una stringa, mettila in una lista
    
    # Controlla se la lista di descrizioni è vuota
    if not descriptions:
        return []  # Restituisce una lista vuota se non ci sono descrizioni
    
    # Genera embedding per tutte le descrizioni in batch
    description_embeddings = generate_sbert_embeddings(descriptions)
    return description_embeddings

def create_multivector_with_svo_and_adj_adv(row, neo4j_handler, model, remove_stopwords=False, lemmatize=False):
    context = row.get('Contesto')
    sentence = row['Frase']
    relazioni_svo = row.get('Relazioni SVO', [])
    aggettivi = row.get('Aggettivi', [])
    avverbi = row.get('Avverbi', [])

    contextual_phrase = context + " " + sentence

    # Embedding della frase
    if remove_stopwords or lemmatize:
        processed_text = remove_stopwords_and_lemmatizer(contextual_phrase, remove_stopwords=remove_stopwords, lemmatize=lemmatize)
        contextual_embedding = generate_sbert_embeddings([processed_text])[0]
    else:
        contextual_embedding = generate_sbert_embeddings([contextual_phrase])[0]

    enriched_embeddings = []
    seen_embeddings = set()  # Set per tracciare gli embedding già aggiunti

    def add_unique_embeddings(embeddings):
        for embedding in embeddings:
            embedding_tuple = tuple(embedding.flatten())  # Converte l'array in una tupla
            if embedding_tuple not in seen_embeddings:
                seen_embeddings.add(embedding_tuple)
                enriched_embeddings.append(embedding)

    # Processare le relazioni SVO con pre-processing su soggetto, verbo, oggetto, predicato
    for relazione in relazioni_svo:
        # Applicare il pre-processing indipendentemente dai flag per soggetto, verbo, oggetto, predicato
        soggetto = remove_stopwords_and_lemmatizer(relazione.get('soggetto', ''), remove_stopwords=True, lemmatize=True)
        verbo = remove_stopwords_and_lemmatizer(relazione.get('verbo', ''), remove_stopwords=True, lemmatize=True)
        oggetto = remove_stopwords_and_lemmatizer(relazione.get('oggetto', ''), remove_stopwords=True, lemmatize=True) if relazione.get('oggetto') else ''
        predicato = remove_stopwords_and_lemmatizer(relazione.get('predicato', ''), remove_stopwords=True, lemmatize=True) if relazione.get('predicato') else ''

        # Ottenere informazioni sugli elementi pre-processati dal grafo
        soggetto_info = neo4j_handler.get_entity_info_from_neo4j(soggetto)
        verbo_info = neo4j_handler.get_verb_info_from_neo4j(verbo)
        oggetto_info = neo4j_handler.get_entity_info_from_neo4j(oggetto) if oggetto else None
        predicato_info = neo4j_handler.get_entity_info_from_neo4j(predicato) if predicato else None

        # Generare embeddings per le descrizioni degli elementi
        soggetto_description_embeddings = process_description_embeddings(soggetto_info.get('description', []), model)
        verbo_description_embeddings = process_description_embeddings(verbo_info.get('description', []), model)
        oggetto_description_embeddings = process_description_embeddings(oggetto_info.get('description', []), model) if oggetto_info else []
        predicato_description_embeddings = process_description_embeddings(predicato_info.get('description', []), model) if predicato_info else []

        # Aggiungi embedding unici
        add_unique_embeddings(soggetto_description_embeddings)
        add_unique_embeddings(verbo_description_embeddings)
        add_unique_embeddings(oggetto_description_embeddings)
        add_unique_embeddings(predicato_description_embeddings)

    # Processare aggettivi e avverbi
    for aggettivo in aggettivi:
        aggettivo_info = neo4j_handler.get_adj_adv_info_from_neo4j(aggettivo)
        aggettivo_description_embeddings = process_description_embeddings(aggettivo_info.get('description', []), model)
        add_unique_embeddings(aggettivo_description_embeddings)

    for avverbio in avverbi:
        avverbio_info = neo4j_handler.get_adj_adv_info_from_neo4j(avverbio)
        avverbio_description_embeddings = process_description_embeddings(avverbio_info.get('description', []), model)
        add_unique_embeddings(avverbio_description_embeddings)

    # Concatenare tutti i vettori arricchiti
    combined_vector = np.concatenate([contextual_embedding, *enriched_embeddings], axis=-1)
    
    return combined_vector

def process_and_save_multivectors(input_file, output_file, neo4j_handler, model, remove_stopwords=False, lemmatize=False):
    print(f"Caricamento del file: {input_file}")
    
    # Caricare il dataset arricchito
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    # Creare un dataframe
    df = pd.DataFrame(data)
    print("Dataframe caricato, avvio elaborazione...")

    # Applicare la funzione di creazione del multivettore a ogni riga del dataset
    df['Multivettore'] = df.apply(lambda row: create_multivector_with_svo_and_adj_adv(row, neo4j_handler, model, remove_stopwords, lemmatize), axis=1)
    
    # Salvare il nuovo dataset con i multivettori inclusi
    df.to_json(output_file, orient='records', indent=4)
    print(f"Multivettori salvati in {output_file}")

In [ ]:
enable_neo4j_population = True

neo4j_handler = Neo4jHandler("bolt://localhost:7687", "neo4j", "10086832")

print("Caricamento del modello Spacy per POS tagging, Dependency Parsing e NER...")
if not 'nlp' in globals():
    nlp = spacy.load("en_core_web_lg")
    print("Modello Spacy caricato.")

# Main logic to load dataset, enrich sentences, and populate Neo4j
if not os.path.exists('stereoset_enriched.json'):

    print("Caricamento del dataset Stereoset (intrasentence)...")
    intrasentence_dataset = load_dataset('McGill-NLP/stereoset', 'intrasentence')
    intersentence_dataset = load_dataset('McGill-NLP/stereoset', 'intersentence')
    print("Dataset Stereoset caricato.")

    data = []
    for i, item in enumerate(intrasentence_dataset['validation']):            
        context = item['context']
        #target = item['target']
        #bias_type = item['bias_type']
        
        for j, sentence in enumerate(item['sentences']['sentence']):
            #gold_label = item['sentences']['gold_label'][j]  
            data.append({
                'Contesto': context,
                'Frase': sentence,
                #'Target': target,
                #'Bias Type': bias_type,
                #'Gold Label': gold_label
            })

    for i, item in enumerate(intersentence_dataset['validation']):            
        context = item['context']
        #target = item['target']
        #bias_type = item['bias_type']
        
        for j, sentence in enumerate(item['sentences']['sentence']):
            #gold_label = item['sentences']['gold_label'][j]  
            data.append({
                'Contesto': context,
                'Frase': sentence,
                #'Target': target,
                #'Bias Type': bias_type,
                #'Gold Label': gold_label
            })
    
    df = pd.DataFrame(data)
    #df = df.head(3)

    print("Estrazione dei ruoli sintattici, NER e relazioni SVO per tutte le frasi...")
    df_enriched = df.apply(lambda row: enrich_sentence(row, enable_neo4j_population), axis=1)
    print("Estrazione completata.")
    
    df_combined = pd.concat([df, df_enriched], axis=1)
    df_combined.to_json('stereoset_enriched.json', orient='records', indent=4)
    print(f"Dataset arricchito con POS, NER, relazioni SVO salvato in 'stereoset_enriched.json'.")

    neo4j_handler.close()
else:
    # Caricamento del modello BERT e tokenizer per l'embedding delle frasi
    #tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    #model = AutoModel.from_pretrained("bert-base-uncased").to("cuda" if torch.cuda.is_available() else "cpu")
    model = SentenceTransformer('paraphrase-mpnet-base-v2')
    
    input_file = 'stereoset_enriched.json'
    print(f"Caricamento del file JSON arricchito 'stereoset_enriched.json'")
    with open(input_file, 'r') as stereoset_file:
        df_final = json.load(stereoset_file)
    
    output_file = 'stereoset_with_multivectors_for_clustering.json'
    process_and_save_multivectors(input_file, output_file, neo4j_handler, model, True, True)
    
    neo4j_handler.close()

Caricamento del modello Spacy per POS tagging, Dependency Parsing e NER...
Modello Spacy caricato.
Caricamento del dataset Stereoset (intrasentence)...
Dataset Stereoset caricato.
Estrazione dei ruoli sintattici, NER e relazioni SVO per tutte le frasi...
Inizio popolamento del database Neo4j per la frase: 'Bengali tigers are beautiful.'
Frase analizzata.
Inizio popolamento del database Neo4j per la frase: 'You can only see them in zoos.'
Nessun synset trovato per il termine 'You'
Nessun synset trovato per il termine 'them'
Frase analizzata.
Inizio popolamento del database Neo4j per la frase: 'There are some left in the wild.'
Frase analizzata.
Inizio popolamento del database Neo4j per la frase: 'He had several assistants with him.'
Frase analizzata.
Inizio popolamento del database Neo4j per la frase: 'The mall closes at ten o'clock.'
Nessun synset trovato per il termine 'ten o'clock'
Frase analizzata.
Inizio popolamento del database Neo4j per la frase: 'He and the defense lawyer were f